# Gesture Recognition - Problem Statement

As a data scientist to a home electronics company which manufactures state of the art smart televisions. We are going to develop a feature in Smart-TV that can recognise five different gestures performed by the user which will help users control the TV without using a remote.

The gestures are continuously monitored by the webcam mounted on the TV. Each gesture corresponds to a specific command:

<br> Thumbs up:  Increase the volume
<br> Thumbs down: Decrease the volume
<br> Left swipe: 'Jump' backwards 10 seconds
<br> Right swipe: 'Jump' forward 10 seconds
<br> Stop: Pause the movie

For analysing videos using neural networks, we are using two most commonly used architectures:

1. 3D Convolutional Network
2. 2D Convolutional Network + RNN ( Using Trannsfer Learning )

In [2]:
import numpy as np
import os
from scipy.misc.pilutil import imread, imresize
import datetime
import os
import cv2

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
from keras.regularizers import l2
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [4]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 25 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
# Parameters initialization
no_of_rows = 120   # X dimension of the image
no_of_cols = 120   # Y dimesnion of the image
no_of_frames = 30  # lenght of the video frames
no_of_channels = 3 # numbe of channels in images 3 for color(RGB)

In [6]:
def crop_image(image):
    x = int(image.shape[0]/2)
    y = int(image.shape[1]/2)
    return image[x-60:x+60,y-60:y+60]


def load_batch_data(source_path, folder_list, batch_size, batch, t, model_type, validation=0):
    if model_type == 'conv3D':
        batch_data = np.zeros((batch_size, no_of_frames, no_of_rows, no_of_cols, no_of_channels)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3
    else:
        batch_data = np.zeros((batch_size, no_of_rows, no_of_cols, no_of_channels))
    #print(batch_data.shape)
    batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
    img_idx = list(range(0,30,1)) #create a list of image numbers you want to use for a particular video
    
    for folder in range(batch_size): # iterate over the batch_size
        # read all the images in the folder
        imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) 
        
        for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
            #print(idx,item)
            image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
               
            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes
            #image = crop_image(image)
            image = cv2.resize(image, (no_of_rows,no_of_cols), interpolation = cv2.INTER_AREA)
            
            normalizedImg = np.zeros((120, 120, 3))
            normalizedImg = cv2.normalize(image, np.zeros((120, 120,3)) , 0, 1, cv2.NORM_MINMAX)
            #print(normalizedImg.shape)
            
            if model_type == 'conv3D':
                batch_data[folder,idx] = normalizedImg #normalise and feed in the image
            else:
                batch_data[folder] = normalizedImg
         
            batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            #print(batch_data.shape)
            
    return (batch_data,batch_labels)


def generator(source_path, folder_list, batch_size, model_type, validation=0):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    
    while True:
        # t contains list of file name
        # 'WIN_20180925_17_44_57_Pro_Thumbs_Down_new;Thumbs_Down_new;3\n'
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            yield load_batch_data(source_path, folder_list, batch_size, batch, t, model_type) #you yield the batch_data and the batch_labels, remember what does yield do
            
        # write the code for the remaining data points which are left after full batches
        remainder = len(folder_list) % batch_size
        if (remainder > 0):
            batch_size = remainder
            yield load_batch_data(source_path, folder_list, batch_size, batch, t, model_type)
            

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [7]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 25
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 25


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

### Conv3D model

In [8]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here

no_of_rows = 120   # X dimension of the image
no_of_cols = 120   # Y dimesnion of the image
no_of_frames = 30  # lenght of the video frames
no_of_channels = 3 # numbe of channels in images 3 for color(RGB)

# Input
input_shape=(no_of_frames, no_of_rows, no_of_cols, no_of_channels)

# Define model
def conv3d_model():
    model = Sequential()

    model.add(Conv3D(8, kernel_size=(3,3,3), input_shape=input_shape, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2,2,2)))

    model.add(Conv3D(16, kernel_size=(3,3,3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2,2,2)))

    model.add(Conv3D(32, kernel_size=(1,3,3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2,2,2)))

    model.add(Conv3D(64, kernel_size=(1,3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(MaxPooling3D(pool_size=(2,2,2)))

    model.add(Flatten())

    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.5))

    model.add(Dense(5, activation='softmax'))
    
    return model


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [9]:
from keras.optimizers import Nadam, Adam, SGD
# optimiser = Adam() #write your optimizer
optimiser = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model = conv3d_model()
model.compile(optimizer=optimiser, loss='mean_squared_error', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)    0         
__________

In [13]:
train_generator = generator(train_path, train_doc, batch_size, 'conv3D')
val_generator = generator(val_path, val_doc, batch_size, 'conv3D')

In [14]:
model_name = 'conv3d_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [15]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [16]:
#batch_size = 10
num_epochs = 25
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 25
Source path =  ./Project_data/train ; batch size = 25
Epoch 1/25


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


27/27 [==============================] - 146s 5s/step - loss: 1.8605 - categorical_accuracy: 0.2395 - val_loss: 1.7280 - val_categorical_accuracy: 0.3300

Epoch 00001: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00001-1.86235-0.23831-1.72796-0.33000.h5
Epoch 2/25
27/27 [==============================] - 31s 1s/step - loss: 1.6615 - categorical_accuracy: 0.2821 - val_loss: 1.5648 - val_categorical_accuracy: 0.2400

Epoch 00002: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00002-1.66145-0.28205-1.56483-0.24000.h5
Epoch 3/25
27/27 [==============================] - 32s 1s/step - loss: 1.4981 - categorical_accuracy: 0.2934 - val_loss: 1.4110 - val_categorical_accuracy: 0.3300

Epoch 00003: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00003-1.49815-0.29345-1.41104-0.33000.h5
Epoch 4/25
27/27 [==============================] - 31s 1s/step - loss: 1.3544 - categorical_accuracy: 0.3248 - val_loss: 1.2800 - val_categorical_accuracy

### Conv3D Model: Saving a copy of the output from each epoch of our last run with SGD optimiser

Source path =  ./Project_data/val ; batch size = 25
Source path =  ./Project_data/train ; batch size = 25
Epoch 1/25
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
27/27 [==============================] - 146s 5s/step - loss: 1.8605 - categorical_accuracy: 0.2395 - val_loss: 1.7280 - val_categorical_accuracy: 0.3300

Epoch 00001: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00001-1.86235-0.23831-1.72796-0.33000.h5
Epoch 2/25
27/27 [==============================] - 31s 1s/step - loss: 1.6615 - categorical_accuracy: 0.2821 - val_loss: 1.5648 - val_categorical_accuracy: 0.2400

Epoch 00002: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00002-1.66145-0.28205-1.56483-0.24000.h5
Epoch 3/25
27/27 [==============================] - 32s 1s/step - loss: 1.4981 - categorical_accuracy: 0.2934 - val_loss: 1.4110 - val_categorical_accuracy: 0.3300

Epoch 00003: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00003-1.49815-0.29345-1.41104-0.33000.h5
Epoch 4/25
27/27 [==============================] - 31s 1s/step - loss: 1.3544 - categorical_accuracy: 0.3248 - val_loss: 1.2800 - val_categorical_accuracy: 0.4600

Epoch 00004: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00004-1.35439-0.32479-1.28004-0.46000.h5
Epoch 5/25
27/27 [==============================] - 31s 1s/step - loss: 1.2195 - categorical_accuracy: 0.3989 - val_loss: 1.1475 - val_categorical_accuracy: 0.4400

Epoch 00005: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00005-1.21952-0.39886-1.14750-0.44000.h5
Epoch 6/25
27/27 [==============================] - 30s 1s/step - loss: 1.1045 - categorical_accuracy: 0.4046 - val_loss: 1.0514 - val_categorical_accuracy: 0.3300

Epoch 00006: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00006-1.10455-0.40456-1.05137-0.33000.h5
Epoch 7/25
27/27 [==============================] - 30s 1s/step - loss: 1.0137 - categorical_accuracy: 0.3533 - val_loss: 0.9498 - val_categorical_accuracy: 0.5400

Epoch 00007: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00007-1.01370-0.35328-0.94981-0.54000.h5
Epoch 8/25
27/27 [==============================] - 32s 1s/step - loss: 0.9148 - categorical_accuracy: 0.4217 - val_loss: 0.8634 - val_categorical_accuracy: 0.5200

Epoch 00008: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00008-0.91483-0.42165-0.86339-0.52000.h5
Epoch 9/25
27/27 [==============================] - 30s 1s/step - loss: 0.8277 - categorical_accuracy: 0.4615 - val_loss: 0.7839 - val_categorical_accuracy: 0.5600

Epoch 00009: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00009-0.82774-0.46154-0.78387-0.56000.h5
Epoch 10/25
27/27 [==============================] - 31s 1s/step - loss: 0.7504 - categorical_accuracy: 0.4843 - val_loss: 0.7118 - val_categorical_accuracy: 0.5800

Epoch 00010: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00010-0.75040-0.48433-0.71184-0.58000.h5
Epoch 11/25
27/27 [==============================] - 31s 1s/step - loss: 0.6871 - categorical_accuracy: 0.4957 - val_loss: 0.6563 - val_categorical_accuracy: 0.4600

Epoch 00011: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00011-0.68709-0.49573-0.65631-0.46000.h5
Epoch 12/25
27/27 [==============================] - 31s 1s/step - loss: 0.6210 - categorical_accuracy: 0.5385 - val_loss: 0.5940 - val_categorical_accuracy: 0.5700

Epoch 00012: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00012-0.62104-0.53846-0.59398-0.57000.h5
Epoch 13/25
27/27 [==============================] - 30s 1s/step - loss: 0.5701 - categorical_accuracy: 0.5214 - val_loss: 0.5373 - val_categorical_accuracy: 0.5900

Epoch 00013: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00013-0.57013-0.52137-0.53735-0.59000.h5
Epoch 14/25
27/27 [==============================] - 30s 1s/step - loss: 0.5233 - categorical_accuracy: 0.5641 - val_loss: 0.5026 - val_categorical_accuracy: 0.5400

Epoch 00014: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00014-0.52332-0.56410-0.50264-0.54000.h5
Epoch 15/25
27/27 [==============================] - 31s 1s/step - loss: 0.4744 - categorical_accuracy: 0.5499 - val_loss: 0.4505 - val_categorical_accuracy: 0.6300

Epoch 00015: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00015-0.47439-0.54986-0.45048-0.63000.h5
Epoch 16/25
27/27 [==============================] - 31s 1s/step - loss: 0.4380 - categorical_accuracy: 0.5698 - val_loss: 0.4213 - val_categorical_accuracy: 0.5500

Epoch 00016: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00016-0.43802-0.56980-0.42131-0.55000.h5
Epoch 17/25
27/27 [==============================] - 29s 1s/step - loss: 0.3980 - categorical_accuracy: 0.6268 - val_loss: 0.3878 - val_categorical_accuracy: 0.6200

Epoch 00017: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00017-0.39800-0.62678-0.38777-0.62000.h5
Epoch 18/25
27/27 [==============================] - 31s 1s/step - loss: 0.3683 - categorical_accuracy: 0.6496 - val_loss: 0.3778 - val_categorical_accuracy: 0.4600

Epoch 00018: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00018-0.36834-0.64957-0.37779-0.46000.h5
Epoch 19/25
27/27 [==============================] - 30s 1s/step - loss: 0.3337 - categorical_accuracy: 0.6524 - val_loss: 0.3253 - val_categorical_accuracy: 0.6300

Epoch 00019: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00019-0.33368-0.65242-0.32530-0.63000.h5
Epoch 20/25
27/27 [==============================] - 31s 1s/step - loss: 0.3044 - categorical_accuracy: 0.6382 - val_loss: 0.2943 - val_categorical_accuracy: 0.7000

Epoch 00020: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00020-0.30436-0.63818-0.29430-0.70000.h5
Epoch 21/25
27/27 [==============================] - 30s 1s/step - loss: 0.2854 - categorical_accuracy: 0.6695 - val_loss: 0.2938 - val_categorical_accuracy: 0.5700

Epoch 00021: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00021-0.28542-0.66952-0.29382-0.57000.h5
Epoch 22/25
27/27 [==============================] - 30s 1s/step - loss: 0.2613 - categorical_accuracy: 0.6724 - val_loss: 0.2477 - val_categorical_accuracy: 0.7300

Epoch 00022: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00022-0.26132-0.67236-0.24768-0.73000.h5
Epoch 23/25
27/27 [==============================] - 30s 1s/step - loss: 0.2422 - categorical_accuracy: 0.7208 - val_loss: 0.2232 - val_categorical_accuracy: 0.7700

Epoch 00023: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00023-0.24223-0.72080-0.22321-0.77000.h5
Epoch 24/25
27/27 [==============================] - 31s 1s/step - loss: 0.2215 - categorical_accuracy: 0.7208 - val_loss: 0.2268 - val_categorical_accuracy: 0.6600

Epoch 00024: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00024-0.22147-0.72080-0.22677-0.66000.h5
Epoch 25/25
27/27 [==============================] - 30s 1s/step - loss: 0.2079 - categorical_accuracy: 0.7179 - val_loss: 0.2038 - val_categorical_accuracy: 0.7100

Epoch 00025: saving model to conv3d_model_init_2019-09-2316_41_06.064134/model-00025-0.20792-0.71795-0.20382-0.71000.h5
<keras.callbacks.History at 0x7f9315178f98>

### Conv3D Model: Saving a copy of the output from each epoch of our last run with Adam optimiser

#batch_size = 10
num_epochs = 25
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 25
Source path =  ./Project_data/train ; batch size = 25
Epoch 1/25
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
27/27 [==============================] - 159s 6s/step - loss: 3.5750 - categorical_accuracy: 0.2438 - val_loss: 2.7970 - val_categorical_accuracy: 0.4200

Epoch 00001: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00001-3.58873-0.24133-2.79698-0.42000.h5
Epoch 2/25
27/27 [==============================] - 34s 1s/step - loss: 2.6460 - categorical_accuracy: 0.3419 - val_loss: 2.3508 - val_categorical_accuracy: 0.5000

Epoch 00002: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00002-2.64597-0.34188-2.35084-0.50000.h5
Epoch 3/25
27/27 [==============================] - 35s 1s/step - loss: 2.3425 - categorical_accuracy: 0.4074 - val_loss: 2.4137 - val_categorical_accuracy: 0.3200

Epoch 00003: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00003-2.34245-0.40741-2.41367-0.32000.h5
Epoch 4/25
27/27 [==============================] - 35s 1s/step - loss: 2.1452 - categorical_accuracy: 0.4046 - val_loss: 2.1684 - val_categorical_accuracy: 0.3300

Epoch 00004: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00004-2.14521-0.40456-2.16838-0.33000.h5
Epoch 5/25
27/27 [==============================] - 41s 2s/step - loss: 1.9346 - categorical_accuracy: 0.4587 - val_loss: 1.7284 - val_categorical_accuracy: 0.5500

Epoch 00005: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00005-1.93456-0.45869-1.72840-0.55000.h5
Epoch 6/25
27/27 [==============================] - 48s 2s/step - loss: 1.7491 - categorical_accuracy: 0.5556 - val_loss: 1.6296 - val_categorical_accuracy: 0.6600

Epoch 00006: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00006-1.74912-0.55556-1.62963-0.66000.h5
Epoch 7/25
27/27 [==============================] - 34s 1s/step - loss: 1.7331 - categorical_accuracy: 0.4900 - val_loss: 1.5372 - val_categorical_accuracy: 0.6600

Epoch 00007: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00007-1.73311-0.49003-1.53718-0.66000.h5
Epoch 8/25
27/27 [==============================] - 32s 1s/step - loss: 1.5444 - categorical_accuracy: 0.5954 - val_loss: 1.8300 - val_categorical_accuracy: 0.4300

Epoch 00008: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00008-1.54438-0.59544-1.83001-0.43000.h5
Epoch 9/25
27/27 [==============================] - 30s 1s/step - loss: 1.4014 - categorical_accuracy: 0.6382 - val_loss: 1.4035 - val_categorical_accuracy: 0.6000

Epoch 00009: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00009-1.40138-0.63818-1.40345-0.60000.h5
Epoch 10/25
27/27 [==============================] - 30s 1s/step - loss: 1.2518 - categorical_accuracy: 0.6809 - val_loss: 1.5591 - val_categorical_accuracy: 0.5400

Epoch 00010: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00010-1.25175-0.68091-1.55907-0.54000.h5
Epoch 11/25
27/27 [==============================] - 30s 1s/step - loss: 1.3287 - categorical_accuracy: 0.6353 - val_loss: 1.3763 - val_categorical_accuracy: 0.6400

Epoch 00011: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00011-1.32866-0.63533-1.37628-0.64000.h5
Epoch 12/25
27/27 [==============================] - 30s 1s/step - loss: 1.1485 - categorical_accuracy: 0.7123 - val_loss: 1.1005 - val_categorical_accuracy: 0.7000

Epoch 00012: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00012-1.14852-0.71225-1.10049-0.70000.h5
Epoch 13/25
27/27 [==============================] - 29s 1s/step - loss: 1.0326 - categorical_accuracy: 0.7379 - val_loss: 1.2343 - val_categorical_accuracy: 0.6500

Epoch 00013: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00013-1.03257-0.73789-1.23435-0.65000.h5
Epoch 14/25
27/27 [==============================] - 30s 1s/step - loss: 1.0529 - categorical_accuracy: 0.7066 - val_loss: 1.9293 - val_categorical_accuracy: 0.4100

Epoch 00014: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00014-1.05288-0.70655-1.92929-0.41000.h5

Epoch 00014: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 15/25
27/27 [==============================] - 30s 1s/step - loss: 0.9412 - categorical_accuracy: 0.7635 - val_loss: 1.6890 - val_categorical_accuracy: 0.4400

Epoch 00015: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00015-0.94120-0.76353-1.68903-0.44000.h5
Epoch 16/25
27/27 [==============================] - 30s 1s/step - loss: 0.9152 - categorical_accuracy: 0.7806 - val_loss: 1.1125 - val_categorical_accuracy: 0.6600

Epoch 00016: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00016-0.91520-0.78063-1.11247-0.66000.h5

Epoch 00016: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 17/25
27/27 [==============================] - 29s 1s/step - loss: 0.7578 - categorical_accuracy: 0.8405 - val_loss: 0.9055 - val_categorical_accuracy: 0.8000

Epoch 00017: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00017-0.75778-0.84046-0.90546-0.80000.h5
Epoch 18/25
27/27 [==============================] - 30s 1s/step - loss: 0.7140 - categorical_accuracy: 0.8604 - val_loss: 0.9037 - val_categorical_accuracy: 0.7900

Epoch 00018: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00018-0.71402-0.86040-0.90366-0.79000.h5
Epoch 19/25
27/27 [==============================] - 29s 1s/step - loss: 0.6763 - categorical_accuracy: 0.8689 - val_loss: 0.8845 - val_categorical_accuracy: 0.7900

Epoch 00019: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00019-0.67626-0.86895-0.88450-0.79000.h5
Epoch 20/25
27/27 [==============================] - 30s 1s/step - loss: 0.7238 - categorical_accuracy: 0.8348 - val_loss: 0.9163 - val_categorical_accuracy: 0.7800

Epoch 00020: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00020-0.72381-0.83476-0.91631-0.78000.h5
Epoch 21/25
27/27 [==============================] - 30s 1s/step - loss: 0.6002 - categorical_accuracy: 0.8860 - val_loss: 0.8814 - val_categorical_accuracy: 0.7900

Epoch 00021: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00021-0.60022-0.88604-0.88137-0.79000.h5
Epoch 22/25
27/27 [==============================] - 29s 1s/step - loss: 0.6390 - categorical_accuracy: 0.8661 - val_loss: 0.8218 - val_categorical_accuracy: 0.8400

Epoch 00022: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00022-0.63904-0.86610-0.82183-0.84000.h5
Epoch 23/25
27/27 [==============================] - 29s 1s/step - loss: 0.6120 - categorical_accuracy: 0.8889 - val_loss: 0.8555 - val_categorical_accuracy: 0.7900

Epoch 00023: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00023-0.61202-0.88889-0.85550-0.79000.h5
Epoch 24/25
27/27 [==============================] - 30s 1s/step - loss: 0.5822 - categorical_accuracy: 0.8889 - val_loss: 0.8740 - val_categorical_accuracy: 0.8200

Epoch 00024: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00024-0.58215-0.88889-0.87397-0.82000.h5

Epoch 00024: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
Epoch 25/25
27/27 [==============================] - 30s 1s/step - loss: 0.5320 - categorical_accuracy: 0.9088 - val_loss: 0.8070 - val_categorical_accuracy: 0.8400

Epoch 00025: saving model to conv3d_model_init_2019-09-2306_16_13.372634/model-00025-0.53202-0.90883-0.80698-0.84000.h5
<keras.callbacks.History at 0x7f2817d5fe80>

### CNN - RNN model

In [8]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout,Reshape
from keras.layers.convolutional import Conv3D, Conv2D, MaxPooling3D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here

no_of_rows1 = 120   # X dimension of the image
no_of_cols1 = 120   # Y dimesnion of the image
no_of_frames1 = 30  # lenght of the video frames
no_of_channels1 = 3 # numbe of channels in images 3 for color(RGB)

# Input
cnn_rnn_input_shape=(no_of_rows1, no_of_cols1, no_of_channels1)

In [9]:
# Transfer Learning with VGGNet
def transfer_vgg16_rnn():
    from keras.applications.vgg16 import VGG16
    from keras.applications.resnet50 import ResNet50
    from keras.models import Model
    from keras.layers import Dense, Input
    from keras.layers.pooling import GlobalAveragePooling2D
    from keras.layers.recurrent import LSTM
    from keras.layers.wrappers import TimeDistributed
   
    
    vgg16_model = VGG16(weights='imagenet',
                               include_top=False,input_shape=cnn_rnn_input_shape)
    model = Sequential()
    
    for layer in vgg16_model.layers:
        #print(layer)
        model.add(layer)
        
    for layer in vgg16_model.layers:
        layer.trainable = False  #Don't train the VGG16 pre-trained layers
     
    
    model.add(Flatten())
    #print (model.summary()) 
    
    #Found out dimensions of flattened vector: 4608 by printing out model.summary()
    model.add(Reshape((4608,1)))
    
    model.add(GRU(128, input_shape=(30, 4608, 1), return_sequences=True, recurrent_regularizer=l2(0.01))) 
    
    model.add(Flatten())

    model.add(Dense(5, activation='softmax'))
 
    
    return model

In [10]:
#from keras.optimizers import Adam
from keras.optimizers import Nadam, Adam, SGD
#optimiser = Adam() #write your optimizer
#optimiser =  Nadam(lr=0.002,
                  #beta_1=0.9,
                  #beta_2=0.999,
                  #epsilon=1e-08,
                  #schedule_decay=0.004)
optimiser1 = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model = transfer_vgg16_rnn()
#model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.compile(loss='mean_squared_error', optimizer=optimiser1, metrics=['categorical_accuracy'])
print (model.summary())

58892288/58889256 [==============================] - 5s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 120, 120, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 120, 120, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 60, 60, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 60, 60, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 60, 60, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 30, 30, 128)       0         
_________________________________________________________________
block3_conv

In [11]:
train_generator1 = generator(train_path, train_doc, batch_size, 'VGG16_RNN') #change it to cnn-rnn for the cnn-rnn model
val_generator1 = generator(val_path, val_doc, batch_size, 'VGG16_RNN') #change it to cnn-rnn for the cnn-rnn model

In [12]:
model_name1 = 'cnn_rnn_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name1):
    os.mkdir(model_name1)
      
    filepath1 = model_name1 + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

    checkpoint1 = ModelCheckpoint(filepath1, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR1 = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list1 = [checkpoint1,LR1]


In [13]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch1 = int(num_train_sequences/batch_size)
else:
    steps_per_epoch1 = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps1 = int(num_val_sequences/batch_size)
else:
    validation_steps1 = (num_val_sequences//batch_size) + 1

In [14]:
#batch_size = 10
num_epochs1 = 30
model.fit_generator(train_generator1, steps_per_epoch=steps_per_epoch1, epochs=num_epochs1, verbose=1, 
                    callbacks=callbacks_list1, validation_data=val_generator1, 
                    validation_steps=validation_steps1, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 25
Source path = Epoch 1/30 ./Project_data/train ; batch size = 25



/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


27/27 [==============================] - 252s 9s/step - loss: 1.4000 - categorical_accuracy: 0.2453 - val_loss: 1.3394 - val_categorical_accuracy: 0.3800

Epoch 00001: val_loss improved from inf to 1.33938, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00001-1.40099-0.24284-1.33938-0.38000.h5
Epoch 2/30
27/27 [==============================] - 204s 8s/step - loss: 1.2776 - categorical_accuracy: 0.3276 - val_loss: 1.2183 - val_categorical_accuracy: 0.2700

Epoch 00002: val_loss improved from 1.33938 to 1.21828, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00002-1.27759-0.32764-1.21828-0.27000.h5
Epoch 3/30
27/27 [==============================] - 208s 8s/step - loss: 1.1560 - categorical_accuracy: 0.3960 - val_loss: 1.0977 - val_categorical_accuracy: 0.4500

Epoch 00003: val_loss improved from 1.21828 to 1.09771, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00003-1.15597-0.39601-1.09771-0.45000.h5
Epoch 4/30
27/27 [=======

27/27 [==============================] - 214s 8s/step - loss: 0.1137 - categorical_accuracy: 0.9259 - val_loss: 0.1655 - val_categorical_accuracy: 0.6600

Epoch 00026: val_loss improved from 0.17171 to 0.16550, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00026-0.11374-0.92593-0.16550-0.66000.h5
Epoch 27/30
27/27 [==============================] - 212s 8s/step - loss: 0.1014 - categorical_accuracy: 0.9430 - val_loss: 0.1591 - val_categorical_accuracy: 0.6300

Epoch 00027: val_loss improved from 0.16550 to 0.15912, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00027-0.10144-0.94302-0.15912-0.63000.h5
Epoch 28/30
27/27 [==============================] - 210s 8s/step - loss: 0.0944 - categorical_accuracy: 0.9402 - val_loss: 0.1459 - val_categorical_accuracy: 0.6700

Epoch 00028: val_loss improved from 0.15912 to 0.14593, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00028-0.09437-0.94017-0.14593-0.67000.h5
Epoch 29/30
27/27 [

### VGG16_RNN Model: Saving a copy of the output from each epoch of our last run 

#batch_size = 10
num_epochs1 = 30
model.fit_generator(train_generator1, steps_per_epoch=steps_per_epoch1, epochs=num_epochs1, verbose=1, 
                    callbacks=callbacks_list1, validation_data=val_generator1, 
                    validation_steps=validation_steps1, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 25
Source path = Epoch 1/30 ./Project_data/train ; batch size = 25

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
27/27 [==============================] - 252s 9s/step - loss: 1.4000 - categorical_accuracy: 0.2453 - val_loss: 1.3394 - val_categorical_accuracy: 0.3800

Epoch 00001: val_loss improved from inf to 1.33938, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00001-1.40099-0.24284-1.33938-0.38000.h5
Epoch 2/30
27/27 [==============================] - 204s 8s/step - loss: 1.2776 - categorical_accuracy: 0.3276 - val_loss: 1.2183 - val_categorical_accuracy: 0.2700

Epoch 00002: val_loss improved from 1.33938 to 1.21828, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00002-1.27759-0.32764-1.21828-0.27000.h5
Epoch 3/30
27/27 [==============================] - 208s 8s/step - loss: 1.1560 - categorical_accuracy: 0.3960 - val_loss: 1.0977 - val_categorical_accuracy: 0.4500

Epoch 00003: val_loss improved from 1.21828 to 1.09771, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00003-1.15597-0.39601-1.09771-0.45000.h5
Epoch 4/30
27/27 [==============================] - 207s 8s/step - loss: 1.0392 - categorical_accuracy: 0.5328 - val_loss: 0.9882 - val_categorical_accuracy: 0.5300

Epoch 00004: val_loss improved from 1.09771 to 0.98820, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00004-1.03923-0.53276-0.98820-0.53000.h5
Epoch 5/30
27/27 [==============================] - 211s 8s/step - loss: 0.9382 - categorical_accuracy: 0.5185 - val_loss: 0.9005 - val_categorical_accuracy: 0.4400

Epoch 00005: val_loss improved from 0.98820 to 0.90051, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00005-0.93821-0.51852-0.90051-0.44000.h5
Epoch 6/30
27/27 [==============================] - 208s 8s/step - loss: 0.8491 - categorical_accuracy: 0.5299 - val_loss: 0.8069 - val_categorical_accuracy: 0.5800

Epoch 00006: val_loss improved from 0.90051 to 0.80687, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00006-0.84911-0.52991-0.80687-0.58000.h5
Epoch 7/30
27/27 [==============================] - 210s 8s/step - loss: 0.7601 - categorical_accuracy: 0.6496 - val_loss: 0.7308 - val_categorical_accuracy: 0.5900

Epoch 00007: val_loss improved from 0.80687 to 0.73075, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00007-0.76006-0.64957-0.73075-0.59000.h5
Epoch 8/30
27/27 [==============================] - 211s 8s/step - loss: 0.6809 - categorical_accuracy: 0.7009 - val_loss: 0.6642 - val_categorical_accuracy: 0.6000

Epoch 00008: val_loss improved from 0.73075 to 0.66420, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00008-0.68090-0.70085-0.66420-0.60000.h5
Epoch 9/30
27/27 [==============================] - 212s 8s/step - loss: 0.6139 - categorical_accuracy: 0.7066 - val_loss: 0.6074 - val_categorical_accuracy: 0.5300

Epoch 00009: val_loss improved from 0.66420 to 0.60745, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00009-0.61392-0.70655-0.60745-0.53000.h5
Epoch 10/30
27/27 [==============================] - 212s 8s/step - loss: 0.5537 - categorical_accuracy: 0.6980 - val_loss: 0.5469 - val_categorical_accuracy: 0.6200

Epoch 00010: val_loss improved from 0.60745 to 0.54688, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00010-0.55374-0.69801-0.54688-0.62000.h5
Epoch 11/30
27/27 [==============================] - 212s 8s/step - loss: 0.5041 - categorical_accuracy: 0.7265 - val_loss: 0.4996 - val_categorical_accuracy: 0.6000

Epoch 00011: val_loss improved from 0.54688 to 0.49965, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00011-0.50407-0.72650-0.49965-0.60000.h5
Epoch 12/30
27/27 [==============================] - 210s 8s/step - loss: 0.4535 - categorical_accuracy: 0.7607 - val_loss: 0.4569 - val_categorical_accuracy: 0.6100

Epoch 00012: val_loss improved from 0.49965 to 0.45693, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00012-0.45348-0.76068-0.45693-0.61000.h5
Epoch 13/30
27/27 [==============================] - 213s 8s/step - loss: 0.4109 - categorical_accuracy: 0.7550 - val_loss: 0.4140 - val_categorical_accuracy: 0.6300

Epoch 00013: val_loss improved from 0.45693 to 0.41399, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00013-0.41089-0.75499-0.41399-0.63000.h5
Epoch 14/30
27/27 [==============================] - 209s 8s/step - loss: 0.3696 - categorical_accuracy: 0.7863 - val_loss: 0.3786 - val_categorical_accuracy: 0.6400

Epoch 00014: val_loss improved from 0.41399 to 0.37856, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00014-0.36963-0.78632-0.37856-0.64000.h5
Epoch 15/30
27/27 [==============================] - 211s 8s/step - loss: 0.3302 - categorical_accuracy: 0.8234 - val_loss: 0.3489 - val_categorical_accuracy: 0.6200

Epoch 00015: val_loss improved from 0.37856 to 0.34887, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00015-0.33016-0.82336-0.34887-0.62000.h5
Epoch 16/30
27/27 [==============================] - 209s 8s/step - loss: 0.3058 - categorical_accuracy: 0.7920 - val_loss: 0.3222 - val_categorical_accuracy: 0.6400

Epoch 00016: val_loss improved from 0.34887 to 0.32223, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00016-0.30582-0.79202-0.32223-0.64000.h5
Epoch 17/30
27/27 [==============================] - 211s 8s/step - loss: 0.2725 - categorical_accuracy: 0.8519 - val_loss: 0.3026 - val_categorical_accuracy: 0.6500

Epoch 00017: val_loss improved from 0.32223 to 0.30265, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00017-0.27248-0.85185-0.30265-0.65000.h5
Epoch 18/30
27/27 [==============================] - 212s 8s/step - loss: 0.2482 - categorical_accuracy: 0.8433 - val_loss: 0.2723 - val_categorical_accuracy: 0.6800

Epoch 00018: val_loss improved from 0.30265 to 0.27232, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00018-0.24820-0.84330-0.27232-0.68000.h5
Epoch 19/30
27/27 [==============================] - 212s 8s/step - loss: 0.2186 - categorical_accuracy: 0.8917 - val_loss: 0.2534 - val_categorical_accuracy: 0.6700

Epoch 00019: val_loss improved from 0.27232 to 0.25341, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00019-0.21858-0.89174-0.25341-0.67000.h5
Epoch 20/30
27/27 [==============================] - 207s 8s/step - loss: 0.2076 - categorical_accuracy: 0.8632 - val_loss: 0.2371 - val_categorical_accuracy: 0.6400

Epoch 00020: val_loss improved from 0.25341 to 0.23707, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00020-0.20759-0.86325-0.23707-0.64000.h5
Epoch 21/30
27/27 [==============================] - 208s 8s/step - loss: 0.1804 - categorical_accuracy: 0.9117 - val_loss: 0.2240 - val_categorical_accuracy: 0.6400

Epoch 00021: val_loss improved from 0.23707 to 0.22399, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00021-0.18036-0.91168-0.22399-0.64000.h5
Epoch 22/30
27/27 [==============================] - 209s 8s/step - loss: 0.1666 - categorical_accuracy: 0.9060 - val_loss: 0.2067 - val_categorical_accuracy: 0.6700

Epoch 00022: val_loss improved from 0.22399 to 0.20671, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00022-0.16658-0.90598-0.20671-0.67000.h5
Epoch 23/30
27/27 [==============================] - 211s 8s/step - loss: 0.1542 - categorical_accuracy: 0.8889 - val_loss: 0.1955 - val_categorical_accuracy: 0.6400

Epoch 00023: val_loss improved from 0.20671 to 0.19551, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00023-0.15417-0.88889-0.19551-0.64000.h5
Epoch 24/30
27/27 [==============================] - 212s 8s/step - loss: 0.1332 - categorical_accuracy: 0.9288 - val_loss: 0.1818 - val_categorical_accuracy: 0.7000

Epoch 00024: val_loss improved from 0.19551 to 0.18178, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00024-0.13320-0.92877-0.18178-0.70000.h5
Epoch 25/30
27/27 [==============================] - 211s 8s/step - loss: 0.1228 - categorical_accuracy: 0.9316 - val_loss: 0.1717 - val_categorical_accuracy: 0.6700

Epoch 00025: val_loss improved from 0.18178 to 0.17171, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00025-0.12277-0.93162-0.17171-0.67000.h5
Epoch 26/30
27/27 [==============================] - 214s 8s/step - loss: 0.1137 - categorical_accuracy: 0.9259 - val_loss: 0.1655 - val_categorical_accuracy: 0.6600

Epoch 00026: val_loss improved from 0.17171 to 0.16550, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00026-0.11374-0.92593-0.16550-0.66000.h5
Epoch 27/30
27/27 [==============================] - 212s 8s/step - loss: 0.1014 - categorical_accuracy: 0.9430 - val_loss: 0.1591 - val_categorical_accuracy: 0.6300

Epoch 00027: val_loss improved from 0.16550 to 0.15912, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00027-0.10144-0.94302-0.15912-0.63000.h5
Epoch 28/30
27/27 [==============================] - 210s 8s/step - loss: 0.0944 - categorical_accuracy: 0.9402 - val_loss: 0.1459 - val_categorical_accuracy: 0.6700

Epoch 00028: val_loss improved from 0.15912 to 0.14593, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00028-0.09437-0.94017-0.14593-0.67000.h5
Epoch 29/30
27/27 [==============================] - 210s 8s/step - loss: 0.0835 - categorical_accuracy: 0.9573 - val_loss: 0.1411 - val_categorical_accuracy: 0.6600

Epoch 00029: val_loss improved from 0.14593 to 0.14111, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00029-0.08351-0.95726-0.14111-0.66000.h5
Epoch 30/30
27/27 [==============================] - 211s 8s/step - loss: 0.0748 - categorical_accuracy: 0.9573 - val_loss: 0.1345 - val_categorical_accuracy: 0.6800

Epoch 00030: val_loss improved from 0.14111 to 0.13452, saving model to cnn_rnn_model_init_2019-09-2309_43_18.886759/model-00030-0.07482-0.95726-0.13452-0.68000.h5
<keras.callbacks.History at 0x7f0b7d176dd8>

### .h5 files link --> https://drive.google.com/drive/folders/1pqSlayViUbcDYeQQfO_qyqTj9cSdZAuK?usp=sharing